In [1]:
from __future__ import division, print_function, absolute_import

import sys
import numpy as np
import tensorflow as tf

from scipy import linalg
from scipy._lib.six import callable, get_method_function, get_function_code
from scipy.special import xlogy

In [2]:
grid_y, grid_x = np.mgrid[0:159:160j, 0:159:160j]
batch_size = 1
landmarks_num = 76
grid_shape = grid_x.shape
tf_xa = np.asarray([a.flatten() for a in [grid_x, grid_y]], dtype=np.float_) # (2, 25600)
tf_xa = np.asarray([tf_xa for _ in range(0, batch_size)], dtype=np.float_) # (batch_size, 2, 25600)

In [3]:
def rbf_tf(pred_x, pred_y, correct_points, grids, batch_size):
    def _euclidean_norm_tf(x1, x2):    
        return tf.reduce_sum(tf.sqrt(((x1 - x2)**2)), 1) 

    def _h_linear_tf(r):
        return r

    def _call_norm_tf(x1, x2):
        x1 = tf.expand_dims(x1, 3) 
        x2 = tf.expand_dims(x2, 2) 
        return _euclidean_norm_tf(x1, x2)    

    # set parameters
    norm = _euclidean_norm_tf
    function = _h_linear_tf
    epsilon = tf.constant(2.)
    smooth = tf.constant(1.)

    xi = tf.concat([tf.reshape(pred_x, (-1, 1, landmarks_num)), tf.reshape(pred_y, (-1, 1, landmarks_num))], 1) # (Node, 2, 76)
    N = xi.shape[-1].value # same as landmarks_num => 76
    di = tf.expand_dims(correct_points, 2) # (?, 76, 1)
        
    r = _call_norm_tf(xi, xi) # (None, 76, 76)
    A = tf.subtract(r, tf.multiply(smooth, tf.eye(N, batch_shape= [batch_size])))

    nodes = tf.matrix_solve (A, di)

    r2 = _call_norm_tf(grids, xi)
    return tf.reshape(tf.matmul(r2, nodes), [-1, grid_shape[0], grid_shape[1]]) 

In [8]:
# test data
correct_x_p = np.random.randint(160, size=landmarks_num)
correct_y_p = np.random.randint(160, size=landmarks_num)
pred_x_p = np.random.randint(160, size=landmarks_num)
pred_y_p = np.random.randint(160, size=landmarks_num)

In [5]:
with tf.Graph().as_default():
    correct_x = tf.placeholder(tf.float32, (None, landmarks_num), name= 'correct_x')
    correct_y = tf.placeholder(tf.float32, (None, landmarks_num), name= 'correct_y')
    pred_x = tf.placeholder(tf.float32, (None, landmarks_num), name= 'pred_x')
    pred_y = tf.placeholder(tf.float32, (None, landmarks_num), name= 'pred_y')
    
#     image_input = tf.constant(image, dtype=tf.float32, name= 'image_input')
    grids = tf.constant(tf_xa, dtype=tf.float32, name= 'grids')
    
    rbf_x = rbf_tf(pred_x, pred_y, correct_x, grids, batch_size)
    rbf_y = rbf_tf(pred_x, pred_y, correct_y, grids, batch_size)
    
    #warped = cv2.remap(image, map_x, map_y, cv2.INTER_LINEAR)
    with tf.Session() as sess :
        sess.run(tf.global_variables_initializer())
        
        feed_dict = {
            correct_x : correct_x_p.reshape(-1, landmarks_num),
            correct_y : correct_y_p.reshape(-1, landmarks_num),
            pred_x : pred_x_p.reshape(-1, landmarks_num),
            pred_y : pred_y_p.reshape(-1, landmarks_num)
        }
        
        x, y = sess.run([rbf_x, rbf_y], feed_dict= feed_dict)

In [7]:
x.shape, y.shape

((1, 160, 160), (1, 160, 160))